In [1]:
#IMPORTS
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import math

from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer,r2_score
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error


#Loading Features
def load_features(path, name):

    image_path = path
    look = name

    desired_files = [file for file in os.listdir(image_path) if not file.startswith('.')]

    features = []
    image_names = []
    count = 0
    for filename in desired_files:
        f = filename[:-4]
        if f[-len(look):] == look:
            feature = np.load(image_path + '/' + filename)
            feature = np.squeeze(feature)
            features.append(feature)
            image_names.append(f[:-len(look)])

    features_df = pd.DataFrame({look : features, 'Image_Name': image_names})

    return features_df

#Converting all hardness values reported as GPa to HV
def Convert_to_HV(gpa, hv):
    if hv == 'N/a':
        return 1000/9.80665*gpa
    else:
        return hv

#Converting all hardness values reported as HV to GPa
def Convert_to_GPA(gpa, hv):
    if gpa == 'N/a':
        return 9.80665/1000*hv
    else:
        return gpa


#Loading the excel sheet containing the corresponding properties and composition information
def load_properties(path):
    properties = pd.read_excel(path)
    for i, col in enumerate(properties.columns):
      print(f"Index {i}: {col}")


    columns_to_combine = properties.iloc[:, 3:25]
    properties['combined_array'] = columns_to_combine.apply(lambda row: row.values.tolist(), axis=1)

    for index, row in properties.iterrows():
      gpa = row['Vickers Hardness (Gpa) ']
      hv = row['Vickers Hardness (HV)']
      properties.at[index, 'Vickers Hardness (HV)'] = Convert_to_HV(gpa, hv)
      properties.at[index, 'Vickers Hardness (Gpa) '] = Convert_to_GPA(gpa, hv)

    return properties


#Calculating MAPE score
def MAPE(y, y_hat):
    mape = np.mean(np.abs((y - y_hat)/y))*100
    return mape

def nested_pr(X, Y, scaled=False):
    # Setup outer cross-validation loop
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)

    # Create the pipeline with PCA and Polynomial Regression
    pca = PCA()
    poly = PolynomialFeatures(degree=2)
    model = LinearRegression()

    if scaled:
        scaler = StandardScaler()
        pipeline = Pipeline([
            ('scaler', scaler),
            ('pca', pca),
            ('poly', poly),
            ('regression', model)
        ])
    else:
        pipeline = Pipeline([
            ('pca', pca),
            ('poly', poly),
            ('regression', model)
        ])

    # Define parameter grid to search for the ideal number of PCs
    param_grid = {
        'pca__n_components': np.arange(1, 13)
    }


    # enumerate splits
    outer_results = list()
    # enumerate splits
    best_pcs = list()
    for train_ix, test_ix in outer_cv.split(X):
        # split data
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = Y[train_ix], Y[test_ix]

        # Nested cross-validation for PCA components tuning
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

        search = GridSearchCV(pipeline, param_grid, cv=inner_cv, scoring='neg_mean_absolute_percentage_error', refit=True)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        best_params = result.best_params_['pca__n_components']
        best_pcs.append(best_params)
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        # evaluate the model
        acc = MAPE(y_test, yhat)
        # store the result
        outer_results.append(acc)

        # report progress
        print('MAPE=%.3f, Train MAPE=%.3f, Number of PCs=%s' % (acc, result.best_score_, best_params))

    most_common_pc = max(set(best_pcs), key=best_pcs.count)
    # summarize the estimated performance of the model
    print(f"\nAverage MAPE across outer folds: {mean(outer_results):.3f}, Standard Deviation: {std(outer_results):.3f}, Most Common PC: {most_common_pc}")
    #print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

    return most_common_pc, outer_results


def nested_lr(X, Y, scaled=False):

    # Setup outer cross-validation loop
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)

    # Create the pipeline with PCA
    pca = PCA()
    model = LinearRegression()

    if scaled:
        scaler = StandardScaler()
        pipeline = Pipeline([
            ('scaler', scaler),
            ('pca', pca),
            ('regression', model)
        ])
    else:
        pipeline = Pipeline([
            ('pca', pca),
            ('regression', model)
        ])

    # Define parameter grid to search for the ideal number of PCs
    param_grid = {
        'pca__n_components': np.arange(1, 51)
    }


    # enumerate splits
    outer_results = list()
    # enumerate splits
    best_pcs = list()
    for train_ix, test_ix in outer_cv.split(X):
        # split data
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = Y[train_ix], Y[test_ix]

        # Nested cross-validation for PCA components tuning
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

        search = GridSearchCV(pipeline, param_grid, cv=inner_cv, scoring='neg_mean_absolute_percentage_error', refit=True)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        best_params = result.best_params_['pca__n_components']
        best_pcs.append(best_params)
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        # evaluate the model
        acc = MAPE(y_test, yhat)
        # store the result
        outer_results.append(acc)

        # report progress
        print('MAPE=%.3f, Train MAPE=%.3f, Number of PCs=%s' % (acc, result.best_score_, best_params))

    most_common_pc = max(set(best_pcs), key=best_pcs.count)
    # summarize the estimated performance of the model
    print(f"\nAverage MAPE across outer folds: {mean(outer_results):.3f}, Standard Deviation: {std(outer_results):.3f}, Most Common PC: {most_common_pc}")
    #print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

    return most_common_pc, outer_results


def nested_svr(X, Y, scaled=False):

    # Setup outer cross-validation loop
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)

    # Create the pipeline with PCA
    pca = PCA()
    svr = SVR(kernel='linear')

    if scaled == True:
        scaler = StandardScaler()
        pipeline = Pipeline([
            ('scaler', scaler),
            ('pca', pca),
            ('svr', svr)
        ])

    else:
        pipeline = Pipeline([
            ('pca', pca),
            ('svr', svr)
        ])


    # Define parameter grid to search for the ideal number of PCs
    param_grid = {
        'pca__n_components': np.arange(1, 51)
    }


    # enumerate splits
    outer_results = list()
    # enumerate splits
    best_pcs = list()
    for train_ix, test_ix in outer_cv.split(X):
        # split data
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = Y[train_ix], Y[test_ix]

        # Nested cross-validation for PCA components tuning
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

        search = GridSearchCV(pipeline, param_grid, cv=inner_cv, scoring='neg_mean_absolute_percentage_error', refit=True)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        best_params = result.best_params_['pca__n_components']
        best_pcs.append(best_params)
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        # evaluate the model
        acc = MAPE(y_test, yhat)
        # store the result
        outer_results.append(acc)

        # report progress
        print('MAPE=%.3f, Train MAPE=%.3f, Number of PCs=%s' % (acc, result.best_score_, best_params))

    most_common_pc = max(set(best_pcs), key=best_pcs.count)
    # summarize the estimated performance of the model
    print(f"\nAverage MAPE across outer folds: {mean(outer_results):.3f}, Standard Deviation: {std(outer_results):.3f}, Most Common PC: {most_common_pc}")
    #print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

    return most_common_pc, outer_results


def Visualization(results, save = False, filename = "", min = 0, max = 50):
    names = ['clip_vitb16', 'clip_vitb32', 'clip_vitl14', 'dino_vits', 'dino_vitb', 'dino_vitl', 'dino_vitg', 'sam', 'nasa']#, 'twopoint']
    medians = np.median(results,axis = 1)

    plt.figure(figsize=(10, 5))

    ax = sns.boxplot(data = results)

    for xtick in ax.get_xticks():
        ax.text(xtick, medians[xtick], f'{medians[xtick]:.2f}',
                    horizontalalignment='center', size='small', color='black')


    ax.set_xticklabels(names)
    ax.set_xlabel('Model')
    ax.set_ylabel('MAPE')
    ax.set_title('2nd Order Polynomial Regression Experimental Results - Just Images')
    ax.set_ylim(min,max)

    plt.tight_layout()
    # if save == True:
    #     plt.savefig(filename, dpi=300)
    plt.show()


def run_svr(x_train, y_train, x,y,num_comp, scale = False):

    if scale == True:
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x = scaler.transform(x)

    pca = PCA(
            svd_solver='full',
            n_components=num_comp,
            random_state=10
        ).fit(x_train)

    x_red = pca.transform(x_train)
    x_red_test = pca.transform(x)

    model = SVR(kernel='linear').fit(x_red,y_train)

    y_hat_train = model.predict(x_red)

    y_hat_train = y_hat_train.reshape(-1,1)

    mape_train = MAPE(y_train, y_hat_train)

    residual_train = y_hat_train-y_train

    y_hat_test = model.predict(x_red_test)

    y_hat_test = y_hat_test.reshape(-1,1)

    residual_test = (y_hat_test-y)

    mape_test = MAPE(y, y_hat_test)

    print ('Train MAPE: %.3f' % (mape_train))
    print ('Test MAPE: %.3f' % (mape_test))


    # Create a square figure
    fig = plt.figure(figsize=(6, 6))  # 6x6 inches square figure

    # Plot data
    x_vals = range(150,750)
    y_vals = range(150,750)
    plt.plot(x_vals,y_vals, color = 'black')
    plt.scatter(y_train,y_hat_train,color="#C1BCB6", zorder = 2)
    plt.scatter(y,y_hat_test,color="#FF206E", zorder = 3) #"#00E632")

    plt.xlabel('True hardness HV', fontsize = 18)
    plt.ylabel('Predicted hardness HV', fontsize = 18)
    plt.xlim(150,750)
    plt.ylim(150,750)
    #plt.ylim(1,7)#200,600)
    # Set equal scaling by setting aspect ratio to 'equal'
    plt.gca().set_aspect('equal', adjustable='box')

    # Save the figure as a PDF file
    name = str(num_comp) + '.pdf'
    plt.savefig(name, format='pdf')

    # Show the plot
    plt.show()





In [ ]:
#Loading the two-point correlations from the matlab files

image_path = 'PATH'
look = '_tp'

desired_files = [file for file in os.listdir(image_path) if not file.startswith('.')]

features = []
image_names = []
count = 0
for filename in desired_files:
    f = filename[:-4]
    if f[-len(look):] == look:
        feature = sio.loadmat(image_path + '/' + filename)
        feature = feature['cross']
        feature = np.array(feature)
        print (feature.shape)
        feature = np.squeeze(feature)
        features.append(feature)
        image_names.append(f[:-len(look)])

tp_df = pd.DataFrame({look : features, 'Image_Name': image_names})

print (tp_df.head())

print (len(tp_df))



In [ ]:
#Loading the all of the other extracted features

names = ['_clip_b16','_clip_b32','_clip_l14','_dino_s','_dino_b','_dino_l','_dino_g','_sam','_Nasa']#,'_twopoint']
path1 = 'FEATURE PATH'
dfs = []
for name in names:
    df1 = load_features(path1, name)
    #df2 = load_features(path2, name)
    #df = pd.concat([df1, df2], axis=0)
    #print (df1.head())
    print (len(df1))
    dfs.append(df1)


In [ ]:
dfs.append(tp_df)
print (len(dfs))

In [ ]:
#Loading the corresponding properties and merging dataframe on the saved image name 

merged_df = load_properties('PROPERTY PATH')

for dataframe in dfs:
    merged_df = pd.merge(dataframe,merged_df, on='Image_Name')

print (merged_df.head())

In [ ]:
clip_vitb16_features = np.array(merged_df['_clip_b16'].tolist())
clip_vitb32_features = np.array(merged_df['_clip_b32'].tolist())
clip_vitl14_features =  np.array(merged_df['_clip_l14'].tolist())
sam_features =  np.array(merged_df['_sam'].tolist())
dino_vits_features =   np.array(merged_df['_dino_s'].tolist())
dino_vitb_features =  np.array(merged_df['_dino_b'].tolist())
dino_vitl_features =  np.array(merged_df['_dino_l'].tolist())
dino_vitg_features =  np.array(merged_df['_dino_g'].tolist())
nasa_features =  np.array(merged_df['_Nasa'].tolist())
twopoint_features = np.array(merged_df['_tp'].tolist())
combined_array = np.array(merged_df['combined_array'].tolist())

In [ ]:
clip_vitb16_features_comp = np.concatenate((clip_vitb16_features, combined_array), axis=1)
clip_vitb32_features_comp = np.concatenate((clip_vitb32_features, combined_array), axis=1)
clip_vitl14_features_comp =  np.concatenate((clip_vitl14_features, combined_array), axis=1)
sam_features_comp = np.concatenate((sam_features, combined_array), axis=1)
dino_vits_features_comp =  np.concatenate((dino_vits_features, combined_array), axis=1)
dino_vitb_features_comp =  np.concatenate((dino_vitb_features, combined_array), axis=1)
dino_vitl_features_comp =  np.concatenate((dino_vitl_features, combined_array), axis=1)
dino_vitg_features_comp =  np.concatenate((dino_vitg_features, combined_array), axis=1)
nasa_features_comp =  np.concatenate((nasa_features, combined_array), axis=1)
twopoint_features_comp = np.concatenate((twopoint_features, combined_array), axis=1)

In [ ]:
y = np.array(merged_df['Vickers Hardness (HV)'].to_list())

In [ ]:
#Running the PR models

print ('Just images')

print ('CLIP VITB16')
clip_vitb16_pr_pc, clip_vitb16_final_mape = nested_pr(clip_vitb16_features, y)

print ('CLIP VITB32')
clip_vitb32_pr_pc, clip_vitb32_final_mape = nested_pr(clip_vitb32_features, y)

print ('CLIP VITL14')
clip_vitl14_pr_pc, clip_vitl14_final_mape = nested_pr(clip_vitl14_features, y)

print ('Dino VITs')
dino_vits_pr_pc, dino_vits_final_mape = nested_pr(dino_vits_features, y)

print ('Dino VITb')
dino_vitb_pr_pc, dino_vitb_final_mape = nested_pr(dino_vitb_features, y)

print ('Dino VITl')
dino_vitl_pr_pc, dino_vitl_final_mape = nested_pr(dino_vitl_features, y)

print ('Dino VITg')
dino_vitg_pr_pc, dino_vitg_final_mape = nested_pr(dino_vitg_features, y)

print ('SAM')
sam_pr_pc, sam_final_mape = nested_pr(sam_features, y)

print ('NASA')
nasa_pr_pc, nasa_final_mape = nested_pr(nasa_features, y)


print ('Images and Compositions')

print ('CLIP VITB16')
clip_vitb16_pr_comp_pc, clip_vitb16_final_mape_comp  = nested_pr(clip_vitb16_features_comp, y, scaled = True)

print ('CLIP VITB32')
clip_vitb32_pr_comp_pc, clip_vitb32_final_mape_comp = nested_pr(clip_vitb32_features_comp, y, scaled = True)

print ('CLIP VITL14')
clip_vitl14_pr_comp_pc, clip_vitl14_final_mape_comp = nested_pr(clip_vitl14_features_comp, y, scaled = True)

print ('Dino VITs')
dino_vits_pr_comp_pc, dino_vits_final_mape_comp = nested_pr(dino_vits_features_comp, y, scaled = True)

print ('Dino VITb')
dino_vitb_pr_comp_pc, dino_vitb_final_mape_comp = nested_pr(dino_vitb_features_comp, y, scaled = True)

print ('Dino VITl')
dino_vitl_pr_comp_pc, dino_vitl_final_mape_comp = nested_pr(dino_vitl_features_comp, y, scaled = True)

print ('Dino VITg')
dino_vitg_pr_comp_pc, dino_vitg_final_mape_comp = nested_pr(dino_vitg_features_comp, y, scaled = True)

print ('SAM')
sam_pr_comp_pc, sam_final_mape_comp = nested_pr(sam_features_comp, y, scaled = True)

print ('NASA')
nasa_pr_comp_pc, nasa_final_mape_comp = nested_pr(nasa_features_comp, y, scaled = True)



In [ ]:
print ('Just images')

tp_pr_pc, tp_final_mape = nested_pr(twopoint_features, y)


print ('Images and Compositions')

tp_pr_comp_pc, tp_final_mape_comp  = nested_pr(twopoint_features_comp, y, scaled = True)

In [ ]:
print ('Just Images')

path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_pr_' + str(tp_pr_pc) + '.npy'
np.save(path, tp_final_mape)

path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_pr_comp_' + str(tp_pr_comp_pc) + '.npy'
np.save(path, tp_final_mape_comp)

In [ ]:
print ('Just Images')

print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_pr_' + str(clip_vitb16_pr_pc) + '.npy'
np.save(path, clip_vitb16_final_mape)

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_pr_' + str(clip_vitb32_pr_pc) + '.npy'
np.save(path, clip_vitb32_final_mape)

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_pr_' + str(clip_vitl14_pr_pc) + '.npy'
np.save(path, clip_vitl14_final_mape)

print ('DINOV2 Small')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_pr_' + str(dino_vits_pr_pc) + '.npy'
np.save(path, dino_vits_final_mape)

print ('DINOV2 Base')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_pr_' + str(dino_vitb_pr_pc) + '.npy'
np.save(path, dino_vitb_final_mape)

print ('DINOV2 Large')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_pr_' + str(dino_vitl_pr_pc) + '.npy'
np.save(path, dino_vitl_final_mape)

print ('DINOV2 Giant')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_pr_' + str(dino_vitg_pr_pc) + '.npy'
np.save(path, dino_vitg_final_mape)

print ('NASA')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_pr_' + str(nasa_pr_pc) + '.npy'
np.save(path, nasa_final_mape)

print ('SAM')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_pr_' + str(sam_pr_pc) + '.npy'
np.save(path, sam_final_mape)

In [ ]:
print ('Just Images')

print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_pr_comp_' + str(clip_vitb16_pr_comp_pc) + '.npy'
np.save(path, clip_vitb16_final_mape_comp)

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_pr_comp_' + str(clip_vitb32_pr_comp_pc) + '.npy'
np.save(path, clip_vitb32_final_mape_comp)

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_pr_comp_' + str(clip_vitl14_pr_comp_pc) + '.npy'
np.save(path, clip_vitl14_final_mape_comp)

print ('DINOV2 Small')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_pr_comp_' + str(dino_vits_pr_comp_pc) + '.npy'
np.save(path, dino_vits_final_mape_comp)

print ('DINOV2 Base')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_pr_comp_' + str(dino_vitb_pr_comp_pc) + '.npy'
np.save(path, dino_vitb_final_mape_comp)

print ('DINOV2 Large')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_pr_comp_' + str(dino_vitl_pr_comp_pc) + '.npy'
np.save(path, dino_vitl_final_mape_comp)

print ('DINOV2 Giant')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_pr_comp_' + str(dino_vitg_pr_comp_pc) + '.npy'
np.save(path, dino_vitg_final_mape_comp)

print ('NASA')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_pr_comp_' + str(nasa_pr_comp_pc) + '.npy'
np.save(path, nasa_final_mape_comp)

print ('SAM')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_pr_comp_' + str(sam_pr_comp_pc) + '.npy'
np.save(path, sam_final_mape_comp)

In [ ]:
pr_results = [clip_vitb16_final_mape,clip_vitb32_final_mape,clip_vitl14_final_mape,dino_vits_final_mape,dino_vitb_final_mape, dino_vitl_final_mape, dino_vitg_final_mape, sam_final_mape,nasa_final_mape]#,tp_pr]
pr_results_comps = [clip_vitb16_final_mape_comp,clip_vitb32_final_mape_comp,clip_vitl14_final_mape_comp,dino_vits_final_mape_comp,dino_vitb_final_mape_comp, dino_vitl_final_mape_comp, dino_vitg_final_mape_comp, sam_final_mape_comp,nasa_final_mape_comp]#,tp_pr_comp]

Visualization(pr_results)
Visualization(pr_results_comps)

In [ ]:
#Running the LR Models

print ('Just images')

print ('CLIP VITB16')
clip_vitb16_lr_pc, clip_vitb16_lr_mape = nested_lr(clip_vitb16_features, y)

print ('CLIP VITB32')
clip_vitb32_lr_pc, clip_vitb32_lr_mape = nested_lr(clip_vitb32_features, y)

print ('CLIP VITL14')
clip_vitl14_lr_pc, clip_vitl14_lr_mape = nested_lr(clip_vitl14_features, y)

print ('Dino VITs')
dino_vits_lr_pc, dino_vits_lr_mape = nested_lr(dino_vits_features, y)

print ('Dino VITb')
dino_vitb_lr_pc, dino_vitb_lr_mape = nested_lr(dino_vitb_features, y)

print ('Dino VITl')
dino_vitl_lr_pc, dino_vitl_lr_mape = nested_lr(dino_vitl_features, y)

print ('Dino VITg')
dino_vitg_lr_pc, dino_vitg_lr_mape = nested_lr(dino_vitg_features, y)

print ('SAM')
sam_lr_pc, sam_lr_mape = nested_lr(sam_features, y)

print ('NASA')
nasa_lr_pc, nasa_lr_mape = nested_lr(nasa_features, y)


print ('Images and Compositions')

print ('CLIP VITB16')
clip_vitb16_lr_comp_pc, clip_vitb16_lr_mape_comp  = nested_lr(clip_vitb16_features_comp, y, scaled = True)

print ('CLIP VITB32')
clip_vitb32_lr_comp_pc, clip_vitb32_lr_mape_comp = nested_lr(clip_vitb32_features_comp, y, scaled = True)

print ('CLIP VITL14')
clip_vitl14_lr_comp_pc, clip_vitl14_lr_mape_comp = nested_lr(clip_vitl14_features_comp, y, scaled = True)

print ('Dino VITs')
dino_vits_lr_comp_pc, dino_vits_lr_mape_comp = nested_lr(dino_vits_features_comp, y, scaled = True)

print ('Dino VITb')
dino_vitb_lr_comp_pc, dino_vitb_lr_mape_comp = nested_lr(dino_vitb_features_comp, y, scaled = True)

print ('Dino VITl')
dino_vitl_lr_comp_pc, dino_vitl_lr_mape_comp = nested_lr(dino_vitl_features_comp, y, scaled = True)

print ('Dino VITg')
dino_vitg_lr_comp_pc, dino_vitg_lr_mape_comp = nested_lr(dino_vitg_features_comp, y, scaled = True)

print ('SAM')
sam_lr_comp_pc, sam_lr_mape_comp = nested_lr(sam_features_comp, y, scaled = True)

print ('NASA')
nasa_lr_comp_pc, nasa_lr_mape_comp = nested_lr(nasa_features_comp, y, scaled = True)


In [ ]:
print ('Just images')

tp_lr_pc, tp_lr_mape = nested_lr(twopoint_features, y)


print ('Images and Compositions')

tp_lr_comp_pc, tp_lr_mape_comp  = nested_lr(twopoint_features_comp, y, scaled = True)

In [ ]:
print ('Just Images')

path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_lr_' + str(tp_lr_pc) + '.npy'
np.save(path, tp_lr_mape)

path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_lr_comp_' + str(tp_lr_comp_pc) + '.npy'
np.save(path, tp_lr_mape_comp)

In [ ]:
print ('Just Images')

print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_lr_' + str(clip_vitb16_lr_pc) + '.npy'
np.save(path, clip_vitb16_lr_mape)

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_lr_' + str(clip_vitb32_lr_pc) + '.npy'
np.save(path, clip_vitb32_lr_mape)

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_lr_' + str(clip_vitl14_lr_pc) + '.npy'
np.save(path, clip_vitl14_lr_mape)

print ('DINOV2 Small')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_lr_' + str(dino_vits_lr_pc) + '.npy'
np.save(path, dino_vits_lr_mape)

print ('DINOV2 Base')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_lr_' + str(dino_vitb_lr_pc) + '.npy'
np.save(path, dino_vitb_lr_mape)

print ('DINOV2 Large')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_lr_' + str(dino_vitl_lr_pc) + '.npy'
np.save(path, dino_vitl_lr_mape)

print ('DINOV2 Giant')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_lr_' + str(dino_vitg_lr_pc) + '.npy'
np.save(path, dino_vitg_lr_mape)

print ('NASA')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_lr_' + str(nasa_lr_pc) + '.npy'
np.save(path, nasa_lr_mape)

print ('SAM')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_lr_' + str(sam_lr_pc) + '.npy'
np.save(path, sam_lr_mape)

In [ ]:
print ('Just Images')

print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_lr_comp_' + str(clip_vitb16_lr_comp_pc) + '.npy'
np.save(path, clip_vitb16_lr_mape_comp)

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_lr_comp_' + str(clip_vitb32_lr_comp_pc) + '.npy'
np.save(path, clip_vitb32_lr_mape_comp)

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_lr_comp_' + str(clip_vitl14_lr_comp_pc) + '.npy'
np.save(path, clip_vitl14_lr_mape_comp)

print ('DINOV2 Small')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_lr_comp_' + str(dino_vits_lr_comp_pc) + '.npy'
np.save(path, dino_vits_lr_mape_comp)

print ('DINOV2 Base')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_lr_comp_' + str(dino_vitb_lr_comp_pc) + '.npy'
np.save(path, dino_vitb_lr_mape_comp)

print ('DINOV2 Large')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_lr_comp_' + str(dino_vitl_lr_comp_pc) + '.npy'
np.save(path, dino_vitl_lr_mape_comp)

print ('DINOV2 Giant')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_lr_comp_' + str(dino_vitg_lr_comp_pc) + '.npy'
np.save(path, dino_vitg_lr_mape_comp)

print ('NASA')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_lr_comp_' + str(nasa_lr_comp_pc) + '.npy'
np.save(path, nasa_lr_mape_comp)

print ('SAM')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_lr_comp_' + str(sam_lr_comp_pc) + '.npy'
np.save(path, sam_lr_mape_comp)

In [ ]:
lr_results = [clip_vitb16_lr_mape,clip_vitb32_lr_mape,clip_vitl14_lr_mape,dino_vits_lr_mape,dino_vitb_lr_mape, dino_vitl_lr_mape, dino_vitg_lr_mape, sam_lr_mape,nasa_lr_mape]#,tp_lr]
lr_results_comps = [clip_vitb16_lr_mape_comp,clip_vitb32_lr_mape_comp,clip_vitl14_lr_mape_comp,dino_vits_lr_mape_comp,dino_vitb_lr_mape_comp, dino_vitl_lr_mape_comp, dino_vitg_lr_mape_comp, sam_lr_mape_comp,nasa_lr_mape_comp]#,tp_lr_comp]

Visualization(lr_results)
Visualization(lr_results_comps)

In [ ]:
#Running the SVR Models

print ('Just images')

print ('CLIP VITB16')
clip_vitb16_svr_pc, clip_vitb16_svr_mape = nested_svr(clip_vitb16_features, y)

print ('CLIP VITB32')
clip_vitb32_svr_pc, clip_vitb32_svr_mape = nested_svr(clip_vitb32_features, y)

print ('CLIP VITL14')
clip_vitl14_svr_pc, clip_vitl14_svr_mape = nested_svr(clip_vitl14_features, y)

print ('Dino VITs')
dino_vits_svr_pc, dino_vits_svr_mape = nested_svr(dino_vits_features, y)

print ('Dino VITb')
dino_vitb_svr_pc, dino_vitb_svr_mape = nested_svr(dino_vitb_features, y)

print ('Dino VITl')
dino_vitl_svr_pc, dino_vitl_svr_mape = nested_svr(dino_vitl_features, y)

print ('Dino VITg')
dino_vitg_svr_pc, dino_vitg_svr_mape = nested_svr(dino_vitg_features, y)

print ('SAM')
sam_svr_pc, sam_svr_mape = nested_svr(sam_features, y)

print ('NASA')
nasa_svr_pc, nasa_svr_mape = nested_svr(nasa_features, y)


print ('Images and Compositions')

print ('CLIP VITB16')
clip_vitb16_svr_comp_pc, clip_vitb16_svr_mape_comp  = nested_svr(clip_vitb16_features_comp, y, scaled = True)

print ('CLIP VITB32')
clip_vitb32_svr_comp_pc, clip_vitb32_svr_mape_comp = nested_svr(clip_vitb32_features_comp, y, scaled = True)

print ('CLIP VITL14')
clip_vitl14_svr_comp_pc, clip_vitl14_svr_mape_comp = nested_svr(clip_vitl14_features_comp, y, scaled = True)

print ('Dino VITs')
dino_vits_svr_comp_pc, dino_vits_svr_mape_comp = nested_svr(dino_vits_features_comp, y, scaled = True)

print ('Dino VITb')
dino_vitb_svr_comp_pc, dino_vitb_svr_mape_comp = nested_svr(dino_vitb_features_comp, y, scaled = True)

print ('Dino VITl')
dino_vitl_svr_comp_pc, dino_vitl_svr_mape_comp = nested_svr(dino_vitl_features_comp, y, scaled = True)

print ('Dino VITg')
dino_vitg_svr_comp_pc, dino_vitg_svr_mape_comp = nested_svr(dino_vitg_features_comp, y, scaled = True)

print ('SAM')
sam_svr_comp_pc, sam_svr_mape_comp = nested_svr(sam_features_comp, y, scaled = True)

print ('NASA')
nasa_svr_comp_pc, nasa_svr_mape_comp = nested_svr(nasa_features_comp, y, scaled = True)


In [ ]:
print ('Just images')

tp_svr_pc, tp_svr_mape = nested_svr(twopoint_features, y)


print ('Images and Compositions')

tp_svr_comp_pc, tp_svr_mape_comp  = nested_svr(twopoint_features_comp, y, scaled = True)

In [ ]:
print ('Just Images')

path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_svr_' + str(tp_svr_pc) + '.npy'
np.save(path, tp_svr_mape)

path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_svr_comp_' + str(tp_svr_comp_pc) + '.npy'
np.save(path, tp_svr_mape_comp)

In [ ]:
print ('Just Images')

print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_svr_' + str(clip_vitb16_svr_pc) + '.npy'
np.save(path, clip_vitb16_svr_mape)

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_svr_' + str(clip_vitb32_svr_pc) + '.npy'
np.save(path, clip_vitb32_svr_mape)

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_svr_' + str(clip_vitl14_svr_pc) + '.npy'
np.save(path, clip_vitl14_svr_mape)

print ('DINOV2 Small')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_svr_' + str(dino_vits_svr_pc) + '.npy'
np.save(path, dino_vits_svr_mape)

print ('DINOV2 Base')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_svr_' + str(dino_vitb_svr_pc) + '.npy'
np.save(path, dino_vitb_svr_mape)

print ('DINOV2 Large')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_svr_' + str(dino_vitl_svr_pc) + '.npy'
np.save(path, dino_vitl_svr_mape)

print ('DINOV2 Giant')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_svr_' + str(dino_vitg_svr_pc) + '.npy'
np.save(path, dino_vitg_svr_mape)

print ('NASA')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_svr_' + str(nasa_svr_pc) + '.npy'
np.save(path, nasa_svr_mape)

print ('SAM')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_svr_' + str(sam_svr_pc) + '.npy'
np.save(path, sam_svr_mape)

In [ ]:
print ('Just Images')

print ('CLIP VITB16')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_svr_comp_' + str(clip_vitb16_svr_comp_pc) + '.npy'
np.save(path, clip_vitb16_svr_mape_comp)

print ('CLIP VITB32')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_svr_comp_' + str(clip_vitb32_svr_comp_pc) + '.npy'
np.save(path, clip_vitb32_svr_mape_comp)

print ('CLIP VITL14')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_svr_comp_' + str(clip_vitl14_svr_comp_pc) + '.npy'
np.save(path, clip_vitl14_svr_mape_comp)

print ('DINOV2 Small')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_svr_comp_' + str(dino_vits_svr_comp_pc) + '.npy'
np.save(path, dino_vits_svr_mape_comp)

print ('DINOV2 Base')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_svr_comp_' + str(dino_vitb_svr_comp_pc) + '.npy'
np.save(path, dino_vitb_svr_mape_comp)

print ('DINOV2 Large')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_svr_comp_' + str(dino_vitl_svr_comp_pc) + '.npy'
np.save(path, dino_vitl_svr_mape_comp)

print ('DINOV2 Giant')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_svr_comp_' + str(dino_vitg_svr_comp_pc) + '.npy'
np.save(path, dino_vitg_svr_mape_comp)

print ('NASA')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_svr_comp_' + str(nasa_svr_comp_pc) + '.npy'
np.save(path, nasa_svr_mape_comp)

print ('SAM')
path = '/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_svr_comp_' + str(sam_svr_comp_pc) + '.npy'
np.save(path, sam_svr_mape_comp)

In [ ]:
svr_results = [clip_vitb16_svr_mape,clip_vitb32_svr_mape,clip_vitl14_svr_mape,dino_vits_svr_mape,dino_vitb_svr_mape, dino_vitl_svr_mape, dino_vitg_svr_mape, sam_lr_mape,nasa_svr_mape]#, tp_svr]
svr_results_comps = [clip_vitb16_svr_mape_comp,clip_vitb32_svr_mape_comp,clip_vitl14_svr_mape_comp,dino_vits_svr_mape_comp,dino_vitb_svr_mape_comp, dino_vitl_svr_mape_comp, dino_vitg_svr_mape_comp, sam_svr_mape_comp,nasa_svr_mape_comp]#, tp_svr_comp]

Visualization(svr_results)
Visualization(svr_results_comps)

In [ ]:
x_0_dino_large, x_test_dino_large, y_0_dino_large, y_test_dino_large = train_test_split(dino_vitl_features,y,test_size=0.1,random_state=2)
x_0_dino_large_comp, x_test_dino_large_comp, y_0_dino_large_comp, y_test_dino_large_comp = train_test_split(dino_vitl_features_comp,y,test_size=0.1,random_state=2)

In [ ]:
run_svr(x_0_dino_large, y_0_dino_large,  x_test_dino_large, y_test_dino_large ,34)

In [ ]:
run_svr(x_0_dino_large_comp, y_0_dino_large_comp,  x_test_dino_large_comp, y_test_dino_large_comp ,49)